In [1]:
# libraries and modules
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Embedding
from keras.layers import LSTM

from keras.optimizers import RMSprop
from keras.optimizers import SGD

from keras import utils as np_utils

import numpy as np
from PIL import Image
import glob
import os
import h5py

# Image Dimensions
# Can very Image sizes, based on computational complexity and accuracy of neural network
IMAGE_WIDTH= 18
IMAGE_HEIGHT = 18
CHANNELS = 3
NUM_CLASSES = 10

# Vary based on accuracy
BATCH_SIZE = 10

Using TensorFlow backend.


In [2]:
# Retrieves images from path

def get_images(path):
    
    images = []
    filenames = glob.glob(path)

    i = 0
    for f in filenames:
        image = Image.open(f)
        image = image.resize((IMAGE_WIDTH,IMAGE_HEIGHT))
        images.append(np.array(image))
        i += 1

    print(i)
    images = np.array(images)
    print(images.shape)
    images = images.reshape(i, IMAGE_WIDTH, IMAGE_HEIGHT, 1)

    images = images.astype('float32')
    images /= 255
    
    return images

In [3]:
# Retrieves images from path
# Inverts images in black and white

def get_images_inverted(path):
    
    images = []
    filenames = glob.glob(path)

    i = 0
    for f in filenames:
        image = Image.open(f)
        image = image.resize((IMAGE_WIDTH,IMAGE_HEIGHT))
        images.append(np.array(image))
        i += 1

    print(i)
    images = np.array(images)
    print(images.shape)
    images = images.reshape(i, IMAGE_WIDTH, IMAGE_HEIGHT, 1)

    images = images.astype('float32')
    images /= 255
    
    images = 1 - images
    
    return images

In [24]:
# Accuracy Function

def accuracy(test_x, test_y, model):
    result = model.predict(test_x)
    predicted_class = np.argmax(result, axis=1)
    true_class = np.argmax(test_y, axis=1)
    num_correct = np.sum(predicted_class == true_class) 
    accuracy = float(num_correct)/result.shape[0]
    return (accuracy * 100)

### CNN Models

In [4]:
# CNN Model 1

def CNN_1():
    
    model = Sequential()
    model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    
    return model

# Add another convolutional layer

def CNN_2():
    
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 1)))
    model.add(Conv2D(32, (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(NUM_CLASSES, activation="softmax"))
    
    return model

# Add Dropout

def CNN_3():
    
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 1)))
    model.add(Conv2D(32, (3,3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.7))
    model.add(Dense(NUM_CLASSES, activation="softmax"))
    
    return model

In [5]:
# Compile Model 1

model1 = CNN_1()

model1.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=['accuracy'])

# Compile Model 2

model2 = CNN_2()

model2.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=['accuracy'])

# Compile Model 3

model3 = CNN_3()

model3.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=['accuracy'])

In [6]:
# Get labels

path = "./digits"

#new path of camera dir
parentDir = os.path.abspath(path)
new_dir = parentDir + "/images/labels"

with open(new_dir) as f:
    labels = f.readlines()

# remove whitespace characters like `\n` at the end of each line
labels = [x.strip() for x in labels]

train_labels = np.array(labels)
train_labels = np_utils.to_categorical(train_labels, NUM_CLASSES)

print(train_labels)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]]


In [7]:
# Get images
images = get_images('./digits/images/*.png')

# Separate training and testing dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, train_labels, test_size=0.2, random_state=42)

10160
(10160, 18, 18)


In [14]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

#show image
img = X_train[0] * 255
img = Image.fromarray(img.reshape(IMAGE_WIDTH, IMAGE_HEIGHT))
img.show()

(8128, 18, 18, 1)
(2032, 18, 18, 1)
(8128, 10)
(2032, 10)


# Notes

### Based on the training session, the validation accuracy stays consistent while the training accuracy increases. This is an exmaple of overfitting, where the model starts to memorize or "learn" the data.


In [15]:
# Model 1 Train
model1.fit(X_train, y_train, validation_split = 0.2, epochs = 10, batch_size = BATCH_SIZE)

Train on 6502 samples, validate on 1626 samples
Epoch 1/10
6502/6502 [==============================] - 9s 1ms/step - loss: 2.3071 - acc: 0.0972 - val_loss: 2.3044 - val_acc: 0.0910
Epoch 2/10
6502/6502 [==============================] - 7s 1ms/step - loss: 2.2995 - acc: 0.1129 - val_loss: 2.3021 - val_acc: 0.1076
Epoch 3/10
6502/6502 [==============================] - 7s 1ms/step - loss: 2.2916 - acc: 0.1269 - val_loss: 2.3045 - val_acc: 0.1046
Epoch 4/10
6502/6502 [==============================] - 7s 1ms/step - loss: 2.2777 - acc: 0.1381 - val_loss: 2.3176 - val_acc: 0.1095
Epoch 5/10
6502/6502 [==============================] - 7s 1ms/step - loss: 2.2570 - acc: 0.1587 - val_loss: 2.3153 - val_acc: 0.1132
Epoch 6/10
6502/6502 [==============================] - 7s 1ms/step - loss: 2.2356 - acc: 0.1678 - val_loss: 2.3324 - val_acc: 0.1039
Epoch 7/10
6502/6502 [==============================] - 7s 1ms/step - loss: 2.2098 - acc: 0.1824 - val_loss: 2.3482 - val_acc: 0.1039
Epoch 8/10
650

In [30]:
# Model 2 Train
model2.fit(X_train, y_train, validation_split = 0.2, epochs = 10, batch_size = BATCH_SIZE)

Train on 6502 samples, validate on 1626 samples
Epoch 1/10
6502/6502 [==============================] - 14s 2ms/step - loss: 2.3051 - acc: 0.0954 - val_loss: 2.3025 - val_acc: 0.1064
Epoch 2/10
6502/6502 [==============================] - 12s 2ms/step - loss: 2.3015 - acc: 0.1086 - val_loss: 2.3063 - val_acc: 0.0947
Epoch 3/10
6502/6502 [==============================] - 12s 2ms/step - loss: 2.2982 - acc: 0.1180 - val_loss: 2.3094 - val_acc: 0.0978
Epoch 4/10
6502/6502 [==============================] - 13s 2ms/step - loss: 2.2889 - acc: 0.1313 - val_loss: 2.3064 - val_acc: 0.1138
Epoch 5/10
6502/6502 [==============================] - 12s 2ms/step - loss: 2.2713 - acc: 0.1467 - val_loss: 2.3278 - val_acc: 0.0972
Epoch 6/10
6502/6502 [==============================] - 12s 2ms/step - loss: 2.2373 - acc: 0.1701 - val_loss: 2.3321 - val_acc: 0.1101
Epoch 7/10
6502/6502 [==============================] - 12s 2ms/step - loss: 2.1895 - acc: 0.1953 - val_loss: 2.3930 - val_acc: 0.1070
Epoch 8

In [38]:
# Model 3 Train
model3.fit(X_train, y_train, validation_split = 0.3, epochs = 10, batch_size = BATCH_SIZE)

Train on 5689 samples, validate on 2439 samples
Epoch 1/10
5689/5689 [==============================] - 13s 2ms/step - loss: 2.3103 - acc: 0.0991 - val_loss: 2.3027 - val_acc: 0.1025
Epoch 2/10
5689/5689 [==============================] - 10s 2ms/step - loss: 2.3026 - acc: 0.1004 - val_loss: 2.3031 - val_acc: 0.0992
Epoch 3/10
5689/5689 [==============================] - 10s 2ms/step - loss: 2.3013 - acc: 0.1107 - val_loss: 2.3039 - val_acc: 0.1029
Epoch 4/10
5689/5689 [==============================] - 10s 2ms/step - loss: 2.3014 - acc: 0.1081 - val_loss: 2.3034 - val_acc: 0.1009
Epoch 5/10
5689/5689 [==============================] - 11s 2ms/step - loss: 2.2988 - acc: 0.1128 - val_loss: 2.3037 - val_acc: 0.1050
Epoch 6/10
5689/5689 [==============================] - 10s 2ms/step - loss: 2.2982 - acc: 0.1139 - val_loss: 2.3047 - val_acc: 0.1000
Epoch 7/10
5689/5689 [==============================] - 10s 2ms/step - loss: 2.2958 - acc: 0.1144 - val_loss: 2.3054 - val_acc: 0.1009
Epoch 8

In [1]:
# Get labels for 2

path = "./digits"

#new path of camera dir
parentDir = os.path.abspath(path)
new_dir = parentDir + "/images2/labels"

with open(new_dir) as f:
    labels_2 = f.readlines()

# remove whitespace characters like `\n` at the end of each line
labels_2 = [x.strip() for x in labels_2]

train_labels_2 = np.array(labels_2)
train_labels_2 = np_utils.to_categorical(train_labels_2, NUM_CLASSES)

print(train_labels_2)

NameError: name 'os' is not defined

In [ ]:
# Get images
images_2 = get_images('./digits/images2/*.png')

# Separate training and testing dataset
from sklearn.model_selection import train_test_split

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(images_2, train_labels_2, test_size=0.2, random_state=42)

In [10]:
# Model 2 Train on different dataset
model2.fit(X_train_2, y_train_2, validation_split = 0.3, epochs = 10, batch_size = BATCH_SIZE)

Train on 5689 samples, validate on 2439 samples
Epoch 1/10
5689/5689 [==============================] - 41s 7ms/step - loss: 2.3052 - acc: 0.1020 - val_loss: 2.3033 - val_acc: 0.0943
Epoch 2/10
5689/5689 [==============================] - 38s 7ms/step - loss: 2.2997 - acc: 0.1213 - val_loss: 2.3071 - val_acc: 0.0984
Epoch 3/10
5689/5689 [==============================] - 38s 7ms/step - loss: 2.2899 - acc: 0.1325 - val_loss: 2.3117 - val_acc: 0.0935
Epoch 4/10
5689/5689 [==============================] - 38s 7ms/step - loss: 2.2591 - acc: 0.1575 - val_loss: 2.3306 - val_acc: 0.0992
Epoch 5/10
5689/5689 [==============================] - 33s 6ms/step - loss: 2.1980 - acc: 0.2007 - val_loss: 2.3810 - val_acc: 0.0923
Epoch 6/10
5689/5689 [==============================] - 33s 6ms/step - loss: 2.0894 - acc: 0.2515 - val_loss: 2.4547 - val_acc: 0.0984
Epoch 7/10
5689/5689 [==============================] - 37s 7ms/step - loss: 1.9463 - acc: 0.3101 - val_loss: 2.5902 - val_acc: 0.0914
Epoch 8

In [72]:
print(accuracy(X_test_2, y_test_2, model2))

9.891732283464567


In [41]:
# Model 3 Train
model3.fit(X_train_2, y_train_2, validation_split = 0.3, epochs = 10, batch_size = BATCH_SIZE)

Train on 5689 samples, validate on 2439 samples
Epoch 1/10
5689/5689 [==============================] - 10s 2ms/step - loss: 2.3085 - acc: 0.1030 - val_loss: 2.3037 - val_acc: 0.0959
Epoch 2/10
5689/5689 [==============================] - 10s 2ms/step - loss: 2.3021 - acc: 0.1048 - val_loss: 2.3044 - val_acc: 0.0894
Epoch 3/10
5689/5689 [==============================] - 10s 2ms/step - loss: 2.3009 - acc: 0.1148 - val_loss: 2.3053 - val_acc: 0.0906
Epoch 4/10
5689/5689 [==============================] - 10s 2ms/step - loss: 2.2983 - acc: 0.1181 - val_loss: 2.3061 - val_acc: 0.0939
Epoch 5/10
5689/5689 [==============================] - 10s 2ms/step - loss: 2.2950 - acc: 0.1194 - val_loss: 2.3078 - val_acc: 0.0873
Epoch 6/10
5689/5689 [==============================] - 10s 2ms/step - loss: 2.2906 - acc: 0.1297 - val_loss: 2.3083 - val_acc: 0.0943
Epoch 7/10
5689/5689 [==============================] - 10s 2ms/step - loss: 2.2817 - acc: 0.1404 - val_loss: 2.3116 - val_acc: 0.0931
Epoch 8

## Small Dataset

### Similar digit and font type to scoreboard digits

In [11]:
# Get labels

path = "./digits"

#new path of camera dir
parentDir = os.path.abspath(path)
new_dir = parentDir + "/small2/labels.txt"

with open(new_dir) as f:
    labels = f.readlines()

# remove whitespace characters like `\n` at the end of each line
labels = [x.strip() for x in labels]

train_labels = np.array(labels)
train_labels = np_utils.to_categorical(train_labels, NUM_CLASSES)

print(train_labels)

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  

In [12]:
# Get images
images = get_images('./digits/small2/*.png')

# Separate training and testing dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, train_labels, test_size=0.2, random_state=42)

78
(78, 18, 18)


In [14]:
# Model 3 Train
model3.fit(X_train, y_train, validation_split = 0.3, epochs = 100, batch_size = BATCH_SIZE)

Train on 43 samples, validate on 19 samples
Epoch 1/100
43/43 [==============================] - 0s 3ms/step - loss: 0.0782 - acc: 0.9767 - val_loss: 0.0072 - val_acc: 1.0000
Epoch 2/100
43/43 [==============================] - 0s 3ms/step - loss: 0.0978 - acc: 0.9767 - val_loss: 0.0059 - val_acc: 1.0000
Epoch 3/100
43/43 [==============================] - 0s 4ms/step - loss: 0.1105 - acc: 0.9767 - val_loss: 0.0068 - val_acc: 1.0000
Epoch 4/100
43/43 [==============================] - 0s 4ms/step - loss: 0.1290 - acc: 0.9767 - val_loss: 0.0116 - val_acc: 1.0000
Epoch 5/100
43/43 [==============================] - 0s 4ms/step - loss: 0.1750 - acc: 0.9302 - val_loss: 0.0057 - val_acc: 1.0000
Epoch 6/100
43/43 [==============================] - 0s 3ms/step - loss: 0.1240 - acc: 0.9767 - val_loss: 0.0069 - val_acc: 1.0000
Epoch 7/100
43/43 [==============================] - 0s 4ms/step - loss: 0.2715 - acc: 0.9070 - val_loss: 0.0126 - val_acc: 1.0000
Epoch 8/100
43/43 [====================

In [16]:
model3.save('digitClassifier.h5')  # creates a HDF5 file

In [17]:
model = load_model('digitClassifier.h5')

In [25]:
#Invert the image because model was trained in white background with black digits

#Test image is from hockey scoreboard, where the background is black and the digit is white
one = get_images_inverted('./digits/one/*.png')

1
(1, 18, 18)


In [26]:
p = model.predict(one, batch_size=32, verbose=1)

1/1 [==============================] - 0s 3ms/step


In [27]:
print(np.argmax(p,1))

[1]
